In [1]:
import numpy as np
from sklearn.decomposition import PCA
import os
from PIL import Image

from sklearn.metrics import pairwise_distances
homePath = r'E:\pycharm files\基于PCA的人脸识别\\'

In [17]:
from CreateDatabase import CreateDatabase
from EigenfaceCore import EigenfaceCore
from Recognition import Recognition

T = CreateDatabase("TrainDatabase\\")
m,A,Eigenfaces = EigenfaceCore(T)
TestImage = r"TestDatabase\1.jpg"
idx=Recognition(TestImage, m, A, Eigenfaces)
print(idx)

E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\1.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\2.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\3.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\4.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\5.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\6.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\7.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\8.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\9.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\10.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\11.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\12.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\13.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\14.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\15.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\16.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\17.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\18.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\19.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\20.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'TestDatabase\\1.jpg'

In [ ]:
# 缩减的写法
def CreateDatabase(TrainDatabase):
    ############# File management
    TrainFiles = os.listdir(TrainDatabase)
    Train_Number = 0;
    # 统计训练图片数量
    for i in range(len(TrainFiles)):
        if TrainFiles[i]!='Thumbs.db':
            Train_Number = Train_Number + 1
    T = []
    for i in range(1,Train_Number+1):
        str_ = homePath+ "TrainDatabase\\" + str(i) + ".jpg"
        print(str_)
        image = Image.open(str_)
        img = list(image.getdata())
        T.append(img)
    T = np.array(T).T
    # print(T)
    return T
T = CreateDatabase(homePath+"TrainDatabase")

In [65]:
# 仿matlab的写法
def CreateDatabase(TrainDatabase):
    ############# File management
    TrainFiles = os.listdir(TrainDatabase)
    ############# Construction of 2D matrix from 1D image vectors
    T = []
    
    for i in TrainFiles:
        if i!='Thumbs.db':
            # Train_Number = Train_Number + 1 # Number of all images in the training database
            str_ = homePath+ "TrainDatabase\\" + i
            image = Image.open(str_)
            img = list(image.getdata())
            T.append(img)
    T = np.array(T).T
    # print(T)
    return T
    ############# Construction of 2D matrix from 1D image vectors
    # T = []
    # for i in range(Train_Number):
    #     str_ = homePath+ "TrainDatabase\\" + str(i) + '.jpg'
    #     image = Image.open(str_)
    #     img = list(image.getdata())
    #     T.append(img)
    # T = np.array(T).T
    # return T



In [34]:
def EigenfaceCore(T):
    
    m = T.mean(axis=1).reshape(-1,1) # Computing the average face image m = (1/P)*sum(Tj's)    (j = 1 : P)
    # Train_Number = T.shape(1)
    
    A = T - m # all centered images
    
    L = np.matmul(A.T,A) # L is the surrogate of covariance matrix C=A*A'.
    values,vectors = np.linalg.eig(L) # Diagonal elements of D are the eigenvalues for both L=A'*A and C=A*A'.

    L_eig_vec = []
    for i in range(len(values)):
        if(values[i]>1):
            L_eig_vec.append(vectors[:,i])
    
    L_eig_vec = np.array(L_eig_vec).T
    Eigenfaces = np.matmul(A,L_eig_vec)
    
    return m,A,Eigenfaces

In [41]:
T = CreateDatabase(homePath+"TrainDatabase")
m,A,Eigenfaces = EigenfaceCore(T)


[[   3.30178814   -8.62034595    0.56996253 ...  -12.51864762
     8.18171614    5.44494558]
 [   3.33939985   -9.16921291    0.33199297 ...  -11.91619206
     8.31500542    4.68863261]
 [   3.50090175  -10.15651638   -0.61809223 ...  -10.87922283
     8.15681302    3.20266331]
 ...
 [-155.50136713  120.25098871 -176.33465409 ...   11.06673549
    -3.05230149  -67.55023589]
 [-164.95715398  104.80606879 -152.10058925 ...    3.36456573
   -29.6650482   -52.92555891]
 [-172.74768403   92.06641195 -143.2323496  ...   10.03725712
   -33.74304885  -61.53974141]]


In [2]:
from CreateDatabase import CreateDatabase
from EigenfaceCore import EigenfaceCore
T = CreateDatabase("TrainDatabase\\")
m,A,Eigenfaces = EigenfaceCore(T)


E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\1.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\2.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\3.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\4.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\5.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\6.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\7.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\8.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\9.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\10.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\11.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\12.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\13.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\14.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\15.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\16.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\17.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\18.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\19.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\20.jpg


In [ ]:
def Recognition(TestImage,m,A,Eigenfaces):
    ProjectedImages = []
    Train_Number = Eigenfaces.shape[1]
    
    
    

In [2]:
Eigenfaces.shape[0]

98304

In [9]:
homePath = r'E:\pycharm files\基于PCA的人脸识别\\'
TestImage = input('Enter test image name (a number between 1 to 10): ')
TestImage = 'TestDatabase/' + TestImage + '.jpg'
image = Image.open(homePath+TestImage)
img = np.array(image.getdata())
img

array([251, 251, 251, ..., 171, 190, 212])

In [4]:
T = CreateDatabase('TrainDatabase\\')
m, A, eigenfaces = EigenfaceCore(T)

E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\1.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\2.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\3.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\4.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\5.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\6.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\7.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\8.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\9.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\10.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\11.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\12.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\13.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\14.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\15.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\16.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\17.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\18.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\19.jpg
E:\pycharm files\基于PCA的人脸识别\\TrainDatabase\20.jpg


In [5]:
ProjectedImages = []
Train_Number = eigenfaces.shape[1]
# 将二十张训练图（centered image）都映射到19维的特征空间中
ProjectedImages = np.matmul(eigenfaces.T,A)

In [14]:
# 读取测试图片
TestImage = 'TestDatabase/7.jpg'
image = Image.open(homePath+TestImage)
InImage = np.array(image.getdata()).reshape(-1,1)
Difference = InImage - m

# 将测试图片（centered image）映射到19维的特征空间中
ProjectedTestImage = np.matmul(eigenfaces.T,Difference)

In [15]:
#计算测试特征图到每一张训练特征图欧氏距离
Euc_dist = []
for i in range(Train_Number):
    q = ProjectedImages[:,i].reshape(1,-1)
    twoPoints =  np.vstack((ProjectedTestImage.T, q))
    temp = pairwise_distances(twoPoints, metric='euclidean')[1,0]
    Euc_dist.append(temp)
print(Euc_dist)
print(np.argmin(Euc_dist)+1)

[1521315932.5216944, 1480970490.2171934, 938506494.4270741, 917734865.1741409, 1580509074.0549664, 1510242739.1136413, 1640583010.266213, 1634880820.3886278, 1603849213.8293357, 1689282892.8862965, 1268132174.6907225, 1472169779.7689495, 384860276.2209874, 94338497.49152769, 1359257452.8890038, 1297902335.1924195, 1733622922.3299952, 1774651475.452517, 1736177262.0303986]
14


In [7]:
def Recognition(TestImage, m, A, eigenfaces):

    Train_Number = eigenfaces.shape[1]
    # 将二十张训练图（centered image）都映射到19维的特征空间中
    ProjectedImages = np.matmul(eigenfaces.T,A)
    # 读取测试图片
    image = Image.open(homePath+TestImage)
    InImage = np.array(image.getdata()).reshape(-1,1)
    # 归一化/中心化
    Difference = InImage - m
    
    # 将测试图片（centered image）映射到19维的特征空间中
    ProjectedTestImage = np.matmul(eigenfaces.T,Difference)
    
    # 计算测试特征图到每一张训练特征图欧氏距离
    Euc_dist = []
    for i in range(Train_Number):
        q = ProjectedImages[:,i].reshape(1,-1)
        twoPoints =  np.vstack((ProjectedTestImage.T, q))
        temp = pairwise_distances(twoPoints, metric='euclidean')[1,0]
        Euc_dist.append(temp)
    out_idx = np.argmin(np.array(Euc_dist))+1
    return out_idx

[array([0, 0]), array([4, 3])]
[[0. 5.]
 [5. 0.]]


In [ ]:
from sklearn.metrics import pairwise_distances
import numpy as np

point1 = np.array([0,0])
point2 = np.array([4, 3])
print([point1, point2])
distance = pairwise_distances([point1, point2], metric='euclidean')
print(distance)

In [ ]:
def Recognition(test_image, m, A, Eigenfaces):
    ProjectedImages = []
    Train_Number = Eigenfaces.shape[1]
    for i in range(Train_Number):
        temp = Eigenfaces.T.dot(A[:,i])
        ProjectedImages.append(temp)
    InputImage = Image.open(test_image).convert('L')
    temp = np.array(InputImage)
    irow, icol = temp.shape
    InImage = temp.reshape(irow*icol, 1)
    Difference = np.double(InImage) - m
    ProjectedTestImage = Eigenfaces.T
    Euc_dist = []
    for i in range(Train_Number):
        q = ProjectedImages[i].reshape(-1, 1)
        temp = np.linalg.norm(ProjectedTestImage - q)**2
        Euc_dist.append(temp)
    Euc_dist_min = min(Euc_dist)
    Recognized_index = np.argmin(Euc_dist)
    OutputName = str(Recognized_index) + '.jpg'
    return OutputName

In [ ]:
OutputName = Recognition(TestImage, m, A, eigenfaces)

selected_image_path = '/data/Disk_A/biancongcong/test/Face/TrainDatabase/' + output_name
print(selected_image_path)
selected_image = Image.open(selected_image_path)

test_image = Image.open(TestImage)
test_image.show()
selected_image.show()

print('Matched image is:', output_name)

